In [3]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

In [4]:
def process_image(img):
    
    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image = image/255.
    image = np.expand_dims(image, axis=0)

    return image

def get_classes(file):
   
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

def draw(image, boxes, scores, classes, all_classes):
    
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))


def detect_image(image, yolo, all_classes):
    
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [6]:
# Detecting various objects in Images

yolo = YOLO(0.4, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

#-----An image having person, car and horse---
f1 = 'person-car-horse.jpg'
path1 = 'images/test/'+f1
image1 = cv2.imread(path1)
image1 = detect_image(image1, yolo, all_classes)
cv2.imwrite('images/res/' + f1, image1)

#-----An image having persons, car and dog---
f2 = 'persons-car-dog.jpg'
path2 = 'images/test/'+f2
image2 = cv2.imread(path2)
image2 = detect_image(image2, yolo, all_classes)
cv2.imwrite('images/res/' + f2, image2)

#----An image having dog, bicycle and person---
f3 = 'dog-bicycle-person.jpg'
path3 = 'images/test/'+f3
image3 = cv2.imread(path3)
image3 = detect_image(image3, yolo, all_classes)
cv2.imwrite('images/res/' + f3, image3)


time: 27.94s
class: person, score: 0.49
box coordinate x,y,w,h: [166.26152396 100.88926944  68.1207329  165.10906559]
class: horse, score: 1.00
box coordinate x,y,w,h: [ 54.58498001 142.51217365 242.32938886 194.35351539]
class: car, score: 0.99
box coordinate x,y,w,h: [256.50203228 192.47191107 220.4093188   75.60168716]
time: 20.56s
class: person, score: 1.00
box coordinate x,y,w,h: [262.1165514   48.7228632  103.76606882 301.12583637]
class: person, score: 0.46
box coordinate x,y,w,h: [382.15705752 178.36940289  14.55458533  36.05409265]
class: dog, score: 0.96
box coordinate x,y,w,h: [370.21049857 259.01699066  96.16820514  90.61459899]
class: car, score: 0.98
box coordinate x,y,w,h: [367.47401953 170.73101997 131.86906278  99.4012773 ]
time: 21.45s
class: person, score: 1.00
box coordinate x,y,w,h: [192.97274947  64.35571611 204.62395251 293.34233701]
class: bicycle, score: 1.00
box coordinate x,y,w,h: [ 38.43025863 112.99792305 349.55060482 295.75549811]
class: dog, score: 1.00
b

True